<a href="https://colab.research.google.com/github/Mohanasandhya1234/FMML-lab/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


answers

1.
Averaging validation accuracy across multiple splits, often referred to as k-fold cross-validation, can indeed provide more consistent and reliable results compared to a single train-test split. This approach helps in reducing the variance in your evaluation and can give you a more robust estimate of your model's performance. Here's why it is beneficial:

Reduced Variance: In a single train-test split, your model's performance can be highly dependent on the specific data points that end up in the training and testing sets. This can lead to high variance in your performance estimate. By using k-fold cross-validation, you average the results over multiple splits, which reduces the impact of random data splitting and provides a more stable evaluation of your model.

Better Generalization: Cross-validation helps you assess how well your model generalizes to unseen data. Averaging performance across multiple folds provides a more accurate estimate of your model's expected performance on new, unseen data.

Utilizes All Data: In k-fold cross-validation, you use all available data for both training and validation, ensuring that every data point contributes to the evaluation of the model's performance. This is especially important when you have a limited dataset.

Identifies Overfitting: Cross-validation can help you identify whether your model is overfitting. If you see a significant discrepancy between training and validation performance across folds, it may indicate overfitting, which can guide you in adjusting your model or regularization techniques.

Robustness to Data Distribution: Cross-validation can provide a more reliable estimate of performance when dealing with imbalanced datasets or datasets with varying data distributions across different classes or categories.

However, it's important to keep in mind a few considerations:

Cross-validation can be computationally expensive, as you're training and evaluating your model multiple times (k times, where k is the number of folds). This may not be practical for very large datasets or complex models.

The choice of k (the number of folds) can affect the results. Common values for k include 5 or 10, but the optimal choice may depend on the size and nature of your dataset.

Ensure that you maintain consistent data preprocessing and feature engineering across all folds to ensure a fair comparison.

In summary, averaging validation accuracy across multiple splits using k-fold cross-validation is a valuable technique for obtaining more consistent and reliable model evaluation results. It helps mitigate the impact of random data splitting and provides a better assessment of your model's generalization performance.


2.
K-fold cross-validation provides a more accurate estimate of how well your model is likely to perform on unseen data compared to a single train-test split. However, it's important to note that the test accuracy estimated through cross-validation is still an estimate, not an exact measure of your model's performance on a completely independent test dataset. Here's why k-fold cross-validation is beneficial:

Reduced Variance: By averaging the performance over multiple folds, cross-validation provides a more stable and less variable estimate of your model's performance. This is especially important when you have limited data, and a single random train-test split can lead to widely varying results.

Better Generalization Assessment: Cross-validation gives you a better understanding of how well your model generalizes to unseen data because it evaluates the model on multiple different subsets of your dataset. This can help you identify if your model is overfitting or underfitting.

Utilizes All Data: Cross-validation ensures that all data points in your dataset are used for both training and validation across the multiple folds. This makes the evaluation more representative of your overall data.

Statistical Significance: Averaging results over multiple folds provides a more statistically significant estimate of your model's performance. This is particularly important when you want to make inferences about your model's performance with a certain level of confidence.

While k-fold cross-validation provides a more accurate estimate of model performance compared to a single train-test split, it's essential to keep in mind that the estimate is still based on your existing dataset. It does not replace the need for a truly independent test dataset, which is often used for final model evaluation.

In practice, it's common to perform k-fold cross-validation to tune hyperparameters, assess model performance, and get an initial sense of how well the model is likely to perform on new, unseen data. After you've iteratively refined your model using cross-validation, you can then use a separate, completely independent test dataset to get a final, unbiased assessment of your model's performance before deploying it in a real-world application.


3.
The number of iterations in k-fold cross-validation can impact the reliability and precision of the estimate of your model's performance. In general, increasing the number of iterations (folds) can provide a more accurate and stable estimate, but it comes with trade-offs in terms of computational cost.

Here's how the number of iterations can affect the estimate:

More Iterations (Folds):

Advantages:
Reduces the impact of random data splitting: With more folds, the influence of the initial random data partitioning is reduced because you're averaging results over multiple splits.
Provides a more stable estimate: Averaging performance over more iterations tends to yield a more stable estimate of your model's performance, which can be particularly beneficial when dealing with small or noisy datasets.
Better assessment of variability: More iterations can help you understand the variability in your model's performance across different subsets of the data.
Disadvantages:
Increased computational cost: Running cross-validation with a higher number of folds requires training and evaluating the model more times, which can be computationally expensive, especially for large datasets or complex models.
Smaller training sets: As you increase the number of folds, the training sets in each fold become smaller, which may limit the ability of your model to capture complex patterns in the data.
Fewer Iterations (Folds):

Advantages:
Lower computational cost: Using fewer folds is computationally less demanding, making it more practical for large datasets and resource-constrained environments.
Larger training sets: With fewer folds, each training set is larger, which can be advantageous when you have limited data.
Disadvantages:
Greater impact of random data splitting: A single random data split can have a more substantial influence on the results, leading to higher variability in your performance estimate.
Less stable estimate: With fewer iterations, the estimate of your model's performance may be less stable and more dependent on the particular data split.
In practice, the choice of the number of iterations (folds) in k-fold cross-validation is often a trade-off between computational cost and the quality of the performance estimate. Common values for k include 5 and 10, but the optimal choice may depend on factors like the size of your dataset, the complexity of your model, and available computational resources.

It's a good practice to start with a reasonable number of folds and then assess whether increasing or decreasing the number of folds has a significant impact on your model evaluation results.


4.
Increasing the number of iterations (folds) in k-fold cross-validation can help mitigate some of the challenges associated with very small training datasets or validation datasets to a certain extent, but it doesn't fundamentally solve the problem of limited data. It can provide more robust estimates of model performance, but there are limitations to what increased iterations can achieve in such scenarios.

Here are some considerations:

Advantages of Increasing Iterations (Folds):

Reduced Impact of Randomness: With more iterations, you reduce the impact of the initial random data splitting. This means you'll have a better sense of how well your model performs across different subsets of the data.

More Stable Estimates: Averaging results over multiple folds tends to yield a more stable estimate of your model's performance, which can be helpful when dealing with small datasets. It can provide a more reliable assessment of your model's generalization ability.

Limitations and Considerations:

Limited Data: If your training dataset is very small, increasing the number of iterations doesn't magically create more training data. Each fold will still have a small training set, which may limit your model's ability to capture complex patterns.

Increased Computational Cost: Running cross-validation with a high number of folds can be computationally expensive, especially if your dataset is small, as it means training and evaluating the model multiple times. This might not be practical in resource-constrained environments.

Overfitting Risk: With very small training sets in each fold, there's a higher risk of overfitting. Models trained on small datasets may perform well on those specific subsets but may not generalize well to new, unseen data.

Biased Estimates: If the dataset is extremely small, even with increased iterations, you might still end up with biased estimates due to the limited diversity in the data.

In cases of extremely small datasets, you should carefully consider the trade-offs between using k-fold cross-validation with more iterations and alternative approaches such as:

Holdout Validation: If you have very limited data, you might consider a simple holdout validation where you split your dataset into a larger portion for training and a smaller portion for validation. While this doesn't provide multiple estimates like k-fold cross-validation, it can be less computationally intensive.

Data Augmentation: If possible, you can apply data augmentation techniques to artificially increase the effective size of your training dataset by generating new training examples from existing ones.

Regularization: Use regularization techniques like dropout or L1/L2 regularization to help prevent overfitting when dealing with small datasets.

In summary, increasing the number of iterations in cross-validation can improve the robustness of your performance estimates, but it doesn't solve the fundamental issue of a very small dataset. You should carefully consider your options and potentially explore techniques beyond cross-validation when dealing with extremely limited data.
